# Solar Flare Analysis: Power-Law Distribution Analysis

This notebook demonstrates how to analyze the power-law distribution of solar flare energies and compare results between traditional flare detection and ML-separated flares.

## Setup and Imports

In [ ]:
%pip install powerlaw

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib import powerlaw

# Add the project root to the path
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

# Import project modules
from config import settings
from src.data_processing.data_loader import load_goes_data, preprocess_xrs_data, remove_background
from src.flare_detection.traditional_detection import (
    detect_flare_peaks, define_flare_bounds, detect_overlapping_flares
)
from src.ml_models.flare_decomposition import FlareDecompositionModel, reconstruct_flares
from src.analysis.power_law import calculate_flare_energy, fit_power_law, compare_flare_populations

## Introduction to Power-Law Distributions

Solar flare energy distributions typically follow a power-law of the form:

$N(E) \propto E^{-\alpha}$

where $N(E)$ is the number of flares with energy $E$, and $\alpha$ is the power-law index (typically around 1.8-2.0 for solar flares).

This distribution indicates that flares exhibit self-similarity across scales, which is characteristic of self-organized criticality (SOC) in complex systems.

In this notebook, we'll analyze the power-law distribution of flare energies using both traditional detection methods and our ML-based flare separation approach.

## Loading and Processing Data

First, let's load and process the GOES XRS data:

In [ ]:
# Locate and load sample data
data_dir = settings.DATA_DIR
sample_files = [f for f in os.listdir(data_dir) if f.endswith('.nc')]

if sample_files:
    data_file = os.path.join(data_dir, sample_files[0])
    print(f"Using {data_file} for demonstration")
    
    # Load and preprocess data
    data = load_goes_data(data_file)
    channel = 'B'  # We'll use the XRS-B channel (0.1-0.8 nm)
    flux_col = f'xrs{channel.lower()}'
    df = preprocess_xrs_data(data, channel=channel, remove_bad_data=True, interpolate_gaps=True)
    
    # Remove background
    df_bg = remove_background(
        df, 
        window_size=settings.BACKGROUND_PARAMS['window_size'],
        quantile=settings.BACKGROUND_PARAMS['quantile']
    )
    
    # Display data sample
    display(df_bg.head())
else:
    print("No .nc files found. Please place GOES XRS data in the 'data' directory.")

## Detecting Flares with Traditional Method

Let's detect flares using the traditional method:

In [ ]:
if 'df' in locals():
    # Detect peaks
    peaks = detect_flare_peaks(
        df, flux_col,
        threshold_factor=settings.DETECTION_PARAMS['threshold_factor'],
        window_size=settings.DETECTION_PARAMS['window_size']
    )
    
    # Define flare bounds
    flares = define_flare_bounds(
        df, flux_col, peaks['peak_index'].values,
        start_threshold=settings.DETECTION_PARAMS['start_threshold'],
        end_threshold=settings.DETECTION_PARAMS['end_threshold'],
        min_duration=settings.DETECTION_PARAMS['min_duration'],
        max_duration=settings.DETECTION_PARAMS['max_duration']
    )
    
    print(f"Detected {len(flares)} flares using traditional method")
    
    # Detect overlapping flares
    overlapping = detect_overlapping_flares(flares, min_overlap='2min')
    print(f"Detected {len(overlapping)} potentially overlapping flare pairs")

## Calculating Flare Energies with Traditional Method

Let's calculate the energy of each flare using the traditional method (without separating overlapping flares):

In [ ]:
if 'flares' in locals() and 'df_bg' in locals():
    # Calculate flare energies
    energy_results_trad = {}
    
    for i, flare in flares.iterrows():
        start_idx = flare['start_index']
        end_idx = flare['end_index']
        
        # Extract the flare segment
        flare_segment = df.iloc[start_idx:end_idx+1].copy()
        
        # Calculate energy
        flare_energy = calculate_flare_energy(
            flare_segment, flux_col, 
            background_column=f'{flux_col}_background' if f'{flux_col}_background' in df_bg.columns else None
        )
        
        # Store the energy
        energy_results_trad[i] = {
            'peak_flux': flare['peak_flux'],
            'integrated_flux': flare_energy['energy'].iloc[-1] if 'energy' in flare_energy.columns else None,
            'duration': flare['duration']
        }
    
    # Create a DataFrame with energy results
    energy_df_trad = pd.DataFrame.from_dict(energy_results_trad, orient='index')
    print("Flare energies from traditional method:")
    display(energy_df_trad.head())
    
    # Create a list of energies for power-law analysis
    traditional_energies = [result['integrated_flux'] for result in energy_results_trad.values() 
                           if result['integrated_flux'] is not None]
    print(f"Number of flares with valid energy measurements: {len(traditional_energies)}")

## Loading ML Model for Flare Separation

Let's load our trained ML model for separating overlapping flares:

In [ ]:
# Initialize the model
model = FlareDecompositionModel(
    sequence_length=settings.ML_PARAMS['sequence_length'],
    n_features=settings.ML_PARAMS['n_features'],
    max_flares=settings.ML_PARAMS['max_flares']
)
model.build_model()

# Try loading the model
model_path = os.path.join(settings.MODEL_DIR, 'flare_decomposition_model')
try:
    model.load_model(model_path)
    print(f"Successfully loaded model from {model_path}")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Training a new model with synthetic data...")
    
    # Generate synthetic training data
    X_train, y_train = model.generate_synthetic_data(n_samples=1000, noise_level=0.05)
    X_val, y_val = model.generate_synthetic_data(n_samples=200, noise_level=0.05)
    
    # Train model
    history = model.train(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,  # Using fewer epochs for demonstration
        batch_size=settings.ML_PARAMS['batch_size'],
        save_path=model_path
    )
    print(f"Model trained and saved to {model_path}")

## Separating Overlapping Flares with ML Model

Now let's use the ML model to separate overlapping flares:

In [ ]:
if 'df' in locals() and 'overlapping' in locals() and len(overlapping) > 0:
    # Process overlapping flares
    ml_results = []
    
    for i, j, duration in overlapping:
        # Get the start and end indices for the overlapping region
        start_idx = min(flares.iloc[i]['start_index'], flares.iloc[j]['start_index'])
        end_idx = max(flares.iloc[i]['end_index'], flares.iloc[j]['end_index'])
        
        # Ensure we have enough context around the flares
        padding = settings.ML_PARAMS['sequence_length'] // 4
        start_idx = max(0, start_idx - padding)
        end_idx = min(len(df) - 1, end_idx + padding)
        
        # Extract the time series segment
        segment = df.iloc[start_idx:end_idx][flux_col].values
        
        # Ensure the segment has the required length for the model
        if len(segment) < settings.ML_PARAMS['sequence_length']:
            # Pad if too short
            segment = np.pad(segment, 
                            (0, settings.ML_PARAMS['sequence_length'] - len(segment)), 
                            'constant')
        elif len(segment) > settings.ML_PARAMS['sequence_length']:
            # Truncate if too long
            segment = segment[:settings.ML_PARAMS['sequence_length']]
        
        # Reshape for model input
        segment = segment.reshape(1, -1, 1)
        
        # Decompose the flares
        original, individual_flares, combined = reconstruct_flares(
            model, segment, window_size=settings.ML_PARAMS['sequence_length'], plot=True
        )
        plt.tight_layout()
        plt.show()
        
        # Store the result
        ml_results.append({
            'overlapping_pair': (i, j),
            'start_idx': start_idx,
            'end_idx': end_idx,
            'original': original.flatten(),
            'individual_flares': individual_flares,
            'combined': combined.flatten()
        })
    
    # Calculate energies for ML-separated flares
    energy_results_ml = {}
    
    for result in ml_results:
        i, j = result['overlapping_pair']
        individual_flares = result['individual_flares']
        
        # For each separated flare
        for k in range(individual_flares.shape[1]):
            if np.max(individual_flares[:, k]) > 0.05 * np.max(result['original']):
                # Calculate the energy using trapezoidal rule
                energy = np.trapz(individual_flares[:, k])
                
                # Store the energy
                energy_results_ml[f"{i}_{j}_{k}"] = {
                    'peak_flux': np.max(individual_flares[:, k]),
                    'integrated_flux': energy,
                    'original_flare': (i, j)
                }
    
    # Create a list of energies for ML-separated flares
    ml_energies = [result['integrated_flux'] for result in energy_results_ml.values() 
                   if result['integrated_flux'] is not None]
    
    print(f"Number of flares after ML separation: {len(ml_energies)}")
else:
    print("No overlapping flares detected or data not available.")

## Power-Law Analysis for Traditional Method

Let's fit a power-law distribution to the flare energies from the traditional method:

In [ ]:
if 'traditional_energies' in locals() and len(traditional_energies) > 0:
    # Fit power law
    powerlaw_trad = fit_power_law(
        traditional_energies,
        xmin=settings.POWERLAW_PARAMS['xmin'],
        xmax=settings.POWERLAW_PARAMS['xmax'],
        n_bootstrap=settings.POWERLAW_PARAMS['n_bootstrap'],
        plot=True
    )
    plt.title('Power-Law Fit for Traditional Method')
    plt.tight_layout()
    plt.show()
    
    print(f"Power-law index (α): {powerlaw_trad['alpha']:.3f} ± {powerlaw_trad['alpha_err']:.3f}")
    print(f"Number of flares used in fit: {powerlaw_trad['n_flares']}")
    print(f"Goodness of fit (R²): {powerlaw_trad['r_squared']:.3f}")
    print(f"p-value: {powerlaw_trad['p_value']:.3e}")
else:
    print("No traditional method energy measurements available.")

## Power-Law Analysis for ML-Separated Flares

Now let's fit a power-law distribution to the flare energies from ML-separated flares:

In [ ]:
if 'ml_energies' in locals() and len(ml_energies) > 0:
    # Update traditional energies by removing overlapping flares and adding separated ones
    modified_energies = traditional_energies.copy()
    
    # Remove the original overlapping flares
    for result in ml_results:
        i, j = result['overlapping_pair']
        if i < len(modified_energies):
            modified_energies[i] = None
        if j < len(modified_energies):
            modified_energies[j] = None
    
    modified_energies = [e for e in modified_energies if e is not None]
    
    # Add the ML-separated flares
    modified_energies.extend(ml_energies)
    
    print(f"Number of flares after removing overlapping and adding separated: {len(modified_energies)}")
    
    # Fit power law for ML-separated flares
    powerlaw_ml = fit_power_law(
        modified_energies,
        xmin=settings.POWERLAW_PARAMS['xmin'],
        xmax=settings.POWERLAW_PARAMS['xmax'],
        n_bootstrap=settings.POWERLAW_PARAMS['n_bootstrap'],
        plot=True
    )
    plt.title('Power-Law Fit for ML-Separated Method')
    plt.tight_layout()
    plt.show()
    
    print(f"Power-law index (α): {powerlaw_ml['alpha']:.3f} ± {powerlaw_ml['alpha_err']:.3f}")
    print(f"Number of flares used in fit: {powerlaw_ml['n_flares']}")
    print(f"Goodness of fit (R²): {powerlaw_ml['r_squared']:.3f}")
    print(f"p-value: {powerlaw_ml['p_value']:.3e}")
else:
    print("No ML-separated flare energy measurements available.")

## Comparing Power-Law Distributions

Let's compare the power-law distributions from both methods:

In [ ]:
if 'powerlaw_trad' in locals() and 'powerlaw_ml' in locals():
    # Compare power-law distributions
    comparison = compare_flare_populations(
        traditional_energies, "Traditional Method",
        modified_energies, "ML-Separated",
        xmin=settings.POWERLAW_PARAMS['xmin'],
        xmax=settings.POWERLAW_PARAMS['xmax'],
        plot=True
    )
    plt.tight_layout()
    plt.show()
    
    print("\nPower-law comparison results:")
    print(f"  Traditional method: α = {powerlaw_trad['alpha']:.3f} ± {powerlaw_trad['alpha_err']:.3f}")
    print(f"  ML-separated method: α = {powerlaw_ml['alpha']:.3f} ± {powerlaw_ml['alpha_err']:.3f}")
    print(f"  Difference: {comparison['alpha_diff']:.3f} ± {comparison['alpha_err_combined']:.3f}")
    print(f"  Significance: {comparison['significance']:.2f}σ")
    print(f"  p-value: {comparison['p_value']:.3e}")
    
    if comparison['p_value'] < 0.05:
        print("\nThe difference in power-law indices is statistically significant.")
        print("This suggests that properly separating overlapping flares affects the measured energy distribution.")
    else:
        print("\nThe difference in power-law indices is not statistically significant.")
        print("This suggests that overlapping flares may not significantly affect the measured energy distribution.")
else:
    print("Insufficient data for comparison.")

## Alternative Power-Law Analysis Methods

Let's also try an alternative method for fitting power-laws using the `powerlaw` package:

In [ ]:
if 'traditional_energies' in locals() and 'modified_energies' in locals():
    try:
        from matplotlib import powerlaw as pl
        
        # Fit traditional energies
        print("Fitting power law to traditional method energies using powerlaw package:")
        fit_trad = pl.Fit(traditional_energies, xmin=settings.POWERLAW_PARAMS['xmin'], xmax=settings.POWERLAW_PARAMS['xmax'])
        print(f"α = {fit_trad.alpha:.3f} ± {fit_trad.sigma:.3f}")
        print(f"xmin = {fit_trad.xmin}")
        print(f"KS statistic = {fit_trad.power_law.KS()}")
        
        # Plot fit
        fig = fit_trad.plot_pdf(linewidth=3)
        fit_trad.power_law.plot_pdf(ax=fig, linestyle='--', color='r', label='Power law fit')
        plt.legend()
        plt.title('Traditional Method - Power-Law Fit')
        plt.tight_layout()
        plt.show()
        
        # Fit ML-separated energies
        print("\nFitting power law to ML-separated energies using powerlaw package:")
        fit_ml = pl.Fit(modified_energies, xmin=settings.POWERLAW_PARAMS['xmin'], xmax=settings.POWERLAW_PARAMS['xmax'])
        print(f"α = {fit_ml.alpha:.3f} ± {fit_ml.sigma:.3f}")
        print(f"xmin = {fit_ml.xmin}")
        print(f"KS statistic = {fit_ml.power_law.KS()}")
        
        # Plot fit
        fig = fit_ml.plot_pdf(linewidth=3)
        fit_ml.power_law.plot_pdf(ax=fig, linestyle='--', color='r', label='Power law fit')
        plt.legend()
        plt.title('ML-Separated Method - Power-Law Fit')
        plt.tight_layout()
        plt.show()
        
        # Compare with other distributions
        print("\nComparing power law with other distributions:")
        distribution_names = ['exponential', 'lognormal', 'stretched_exponential', 'truncated_power_law']
        for dist_name in distribution_names:
            R, p = fit_trad.distribution_compare('power_law', dist_name)
            print(f"Traditional vs {dist_name}: R={R:.3f}, p={p:.3e}")
            R, p = fit_ml.distribution_compare('power_law', dist_name)
            print(f"ML-separated vs {dist_name}: R={R:.3f}, p={p:.3e}")
            print()
    except ImportError:
        print("The 'powerlaw' package is not installed. Please install with 'pip install powerlaw'.")

## Implications for Solar Physics

Let's discuss the implications of our findings for solar physics:

The power-law distribution of solar flare energies is often associated with self-organized criticality (SOC) in the solar corona. The power-law index α has physical significance:

- For α ≈ 2.0: Energy is roughly evenly distributed across different scales
- For α > 2.0: Energy release is dominated by smaller flares ("nanoflare heating")
- For α < 2.0: Energy release is dominated by larger flares

If separating overlapping flares significantly changes the power-law index, it could affect our understanding of:

1. **Solar Coronal Heating**: The mechanism for heating the solar corona to millions of degrees
2. **Flare Prediction**: Statistical models for predicting large flares
3. **Solar-Terrestrial Effects**: Assessment of space weather impacts on Earth

Our analysis helps determine whether properly separating overlapping flares is essential for accurate power-law analysis, or if traditional methods provide sufficient accuracy for these purposes.

## Summary of Results

In this notebook, we've demonstrated:

1. How to calculate flare energies using both traditional detection and ML-based flare separation
2. How to fit power-law distributions to flare energy data using multiple methods
3. How to compare power-law distributions to assess statistical significance of differences
4. The potential implications of our findings for solar physics

Key findings:
- Traditional method power-law index: α = [Result from your analysis]
- ML-separated method power-law index: α = [Result from your analysis]
- Statistical significance of difference: [Result from your analysis]

This analysis contributes to our understanding of solar flare energy distributions and the importance of properly handling overlapping flares in statistical studies.